In [2]:
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark import SparkContext
runModel = DecisionTreeModel.load(sc, "run_regressor/tmp/myDecisionTreeClassificationModel")
wktsModel = DecisionTreeModel.load(sc, "wickets_regressor/tmp/myDecisionTreeClassificationModel")

In [36]:
import pandas as pd
import random
import csv
from numpy import array
match_data = pd.read_csv("/home/mehul/Downloads/New_BData/matches/match4.csv")
match_data.head()

,T1 batsman,T1 bowler,T2 batsman,T2 bowler
0,AM Rahane,DS Kulkarni,WP Saha,B Kumar
1,DJM Short,JD Unadkat,S Dhawan,B Stanlake
2,SV Samson,S Gopal,KS Williamson,Shakib Al Hasan
3,BA Stokes,B Laughlin,MK Pandey,S Kaul
4,RA Tripathi,DJM Short,DJ Hooda,Rashid Khan


In [37]:
T1_batsman = match_data['T1 batsman']
T1_bowler = match_data['T1 bowler']
T2_batsman = match_data['T2 batsman']
T2_bowler = match_data['T2 bowler']


In [38]:
scores = [0, 1, 2, 3, 4, 5, 6]

In [39]:
#venue = str(input('Input Venue of Match'))

In [40]:
def get_cluster(batsman,bowler,non_striker):
    bat_cluster_data = pd.read_csv("/home/mehul/Downloads/New_BData/ML Data/MLbat_p_c.csv",header = 0).drop_duplicates()
    bowl_cluster_data = pd.read_csv("/home/mehul/Downloads/New_BData/ML Data/MLbowl_p_c.csv",header = 0).drop_duplicates()
    debu_bat_cluster_data = pd.read_csv("/home/mehul/Downloads/New_BData/ML Data/MLdebu_bat_p_c.csv",header = 0).drop_duplicates()
    debu_bowl_cluster_data = pd.read_csv("/home/mehul/Downloads/New_BData/ML Data/MLdebu_bowl_p_c.csv",header = 0).drop_duplicates()
    
    bat1_cluster = bat_cluster_data.loc[bat_cluster_data['batsman']==batsman]
    if((bat1_cluster.empty)==True):
        bat1_cluster = debu_bat_cluster_data.loc[debu_bat_cluster_data['batsman']==batsman]
    #print(bat1_cluster)
    
    
    bowl_cluster = bowl_cluster_data.loc[bowl_cluster_data['bowler']==bowler]
    if((bowl_cluster.empty)==True):
        bowl_cluster = debu_bowl_cluster_data.loc[debu_bowl_cluster_data['bowler']==bowler]
    #print(bowl_cluster)    
    
    #print(non_striker)
    bat2_cluster = bat_cluster_data.loc[bat_cluster_data['batsman']==non_striker]
    if((bat2_cluster.empty)==True):
        bat2_cluster = debu_bat_cluster_data.loc[debu_bat_cluster_data['batsman']==non_striker]
    #bat1_profile = array([bat1_cluster['bat_cluster'].item(),bat1_cluster['Avg'].item(),bat1_cluster['SR'].item()])
    #print(bat2_cluster)
    
    
    return array([bat1_cluster['bat_cluster'].item(),bat1_cluster['Avg'].item(),bat1_cluster['SR'].item(),bowl_cluster['bowl_cluster'].item(),bowl_cluster['Avg'].item(),bowl_cluster['SR'].item(),bowl_cluster['Econ'].item(),bat2_cluster['bat_cluster'].item(),bat2_cluster['Avg'].item(),bat2_cluster['SR'].item()])

In [41]:
print(runModel.predict(get_cluster(T1_batsman[0],T2_bowler[1],T1_batsman[1])))
print(wktsModel.predict(get_cluster(T1_batsman[0],T2_bowler[1],T1_batsman[1])))

1.0
0.0


In [42]:
#check,pvp_data = pvp_profile(T1_batsman[1],T2_bowler[0])

In [43]:
#print(pvp_data)

In [44]:
#cvc_data = cvc_profile(bat_cluster,bowl_cluster)

In [45]:
#print(cvc_data)

In [46]:

def match(bat_order, bow_order,inn) :
    tot_wickets = 0
    striker = 1
    non_striker = 0
    striker_notout = 1
    non_striker_notout = 1
    bow_index_order = [0,1,0,1,2,4,3,2,3,4,2,3,4,1,3,4,0,2,0,1]#bowling order
    x = bow_index_order[0]

    total_runs = 0
    k = -1
    print(bat_order[non_striker].rstrip())
    print(bat_order[striker].rstrip())
    for i in range(0,120) :
        # Swap batsmen and Change bowlers after every over
        score_sheet = [0]*5
        if i%6 == 0 :
            k += 1
            x = bow_index_order[k]

            tmp_striker = striker
            tmp_notout = striker_notout
            striker = non_striker
            striker_notout = non_striker_notout
            non_striker = tmp_striker
            non_striker_notout = tmp_notout

        curr_bat = bat_order[striker].rstrip() #Striker
        #print(curr_bat)
        score_sheet[0] = curr_bat
        other_bat = bat_order[non_striker].rstrip()# Non Striker
        #print(other_bat)
        score_sheet[1] = other_bat
        curr_bow = bow_order[x].rstrip()
        score_sheet[2] = curr_bow

        #Prediction
        
        data = get_run_cluster(curr_bat,curr_bow,other_bat)
        run_prediction = round(runModel.predict(data))
        wkt_prediction = wktsModel.predict(data)
        #if Out
        if(wkt_prediction >0.15):
            tot_wickets+=1
            striker=max(striker,non_striker) + 1
            score_sheet[4] = 1
            print(bat_order[striker].rstrip())
            striker_notout = 1

        if striker > 10:
            break
            
        elif run_prediction==0 or run_prediction==2 or run_prediction==4 or run_prediction==6:
            total_runs+=run_prediction


        elif run_prediction==1 or run_prediction==3:
            total_runs+=run_prediction
            tmp_striker = striker
            tmp_notout = striker_notout
            striker = non_striker
            striker_notout = non_striker_notout
            non_striker = tmp_striker
            non_striker_notout = tmp_notout



        if inn == 1 :
            global first_inn_score
            first_inn_score = total_runs

        # If it is second innings and if the team has chased down the target
        if inn == 2 and total_runs > first_inn_score :
            break
        #print(score_sheet) #UNCOMMENT TO PRINT BALL BY BALL STATS

    num_of_overs_played = str(int((i+1)/6)) + "." + str((i+1)%6)
    return total_runs, tot_wickets, str(total_runs)+"/"+str(tot_wickets)+" Overs : "+ num_of_overs_played


In [47]:
first_innings_score, wickets1, formatted_score1 = match(T1_batsman, T2_bowler, 1)
print ("Team 1 Score : " + formatted_score1)
print("\n*****************************\n")

second_innings_score, wickets2, formatted_score2 = match(T2_batsman, T1_bowler, 2)
print ("Team 2 Score : " + formatted_score2)

if first_innings_score > second_innings_score :
    print ("Team 1 wins!")
    print ("By "+str(first_innings_score - second_innings_score)+" runs")
elif second_innings_score > first_innings_score :
    print ("Team 2 wins!")
    print ("By "+ str(10 - wickets2) +" wickets")
else :
    print ("Match Tied.")

AM Rahane
DJM Short
SV Samson
BA Stokes
RA Tripathi
Team 1 Score : 91/3 Overs : 20.0

*****************************

WP Saha
S Dhawan
KS Williamson
MK Pandey
DJ Hooda
Team 2 Score : 94/3 Overs : 5.3
Team 2 wins!
By 7 wickets


In [35]:
data = [0,2,3,26]

In [ ]:
sameModel.predict(data)